In [120]:
# from types import MethodType # i dont think you need this at all

p = 13
K = GF(p)
n = 5
m = 4
R = PolynomialRing(K, x, n) # defines n variables over K
R.inject_variables(verbose=False) # makes all the variables ready for use

In [118]:
# generate two m-tuples of inhomogenous multivariate polynomials of degree 2,

F = []
P = []
for i in range(m):
    F.append(R.random_element(degree=2))
    P.append(R.random_element(degree=2))

f = vector(F)
p = vector(P)

In [124]:
# class QuadraticForm(SageObject):
# adjusted Gram matrix method that works for any ring of char != 2
def my_Gram_matrix(self):
    R = self.base_ring()
    assert R.characteristic() != 2, "Characteristic of the base ring cannot be 2."
    A = (R(1) / R(2)) * self.matrix()
    n = self.dim()

    # Test to see if it has an integral Gram matrix
    Int_flag = True
    for i in range(n):
        for j in range(i, n):
            Int_flag &= A[i, j] in R

    # Return the Gram matrix, or an error
    if Int_flag:
        return MatrixSpace(R, n, n)(A)
    raise TypeError("this form does not have an integral Gram matrix")

QuadraticForm.my_Gram_matrix = my_Gram_matrix

In [122]:
# given a polynomial f, output the matrix A, vector B and scalar C s.t.
# f(x) = x^T Ax + Bx + C
def quadratic_to_matrix(f):
    h = f.homogeneous_components()
    A = QuadraticForm(h.get(2)).my_Gram_matrix()
    B = vector([])
    C = 0

    # try/except but for the poor ppl 
    if h.get(1) != None:
        # we make a vector of the coefficients of each basis element
        B = vector([h.get(1).coefficient(R.gens()[i]) for i in range(n)])

    if h.get(0) != None:
        C = h.get(0)

    return A, B, C

In [113]:
# a function that simulates precomposition of a quadratic form
# f(x) = x^T Ax + Bx + C with a n x n matrix S 
# so it returns f \circ S 
def transform_input(S, A, B, C):
    # this throws an error because S^T A S is a matrix but BS is a vector and you cannot
    # put them in together in one vector
    #
    # it works if you return them separately, not in a vector
    return vector(K, [S.transpose()*A*S, B*S, C])

In [86]:
# We define the ring with enough variables for both our matrices
R2  = PolynomialRing(K, n^2 + m^2, 'y')

# first n^2 elements of R2
S = matrix(R2, n, n, R2.gens()[:n^2])

# we get the last m^2 variables of R2
T = matrix(R2, m, m, R2.gens()[-m^2:])
print(S, T, A, B, C)

[ y0  y1  y2  y3  y4]
[ y5  y6  y7  y8  y9]
[y10 y11 y12 y13 y14]
[y15 y16 y17 y18 y19]
[y20 y21 y22 y23 y24] [y25 y26 y27 y28]
[y29 y30 y31 y32]
[y33 y34 y35 y36]
[y37 y38 y39 y40] [ 0  0  0  0  1]
[ 0  0  0  3  0]
[ 0  0  0  0  0]
[ 0  3  0  0 12]
[ 1  0  0 12  0] (0, 0, 5, 0, 0) -5


In [114]:
def random_symmetric_matrix(K, n):
    # get a random matrix
    A = random_matrix(K, n, n)
    # and make it symmetric
    return (A + A.transpose()) / 2

# returns A, B and C like above
def random_quadratic_poly(K, n):
    return random_symmetric_matrix(K, n), random_vector(K, n), K.random_element()

In [117]:
# an actual example
# instead of generating an actual polynomial first 
# and then getting the corresponding matrices,
# generate the matrices randomly

# here are the actual transformations
T = matrix(GL(m, K).random_element())
S = matrix(GL(n, K).random_element())

# generate a whole ass m-sized vector of polynomials
F = zero_matrix(K, m, 3)
# for i in range(m):

i = 0
A, B, C = random_quadratic_poly(K, n)
print(transform_input(S, A, B, C))
    
print(F[1])

TypeError: unable to convert [ 5  4  5 12  4]
[ 4 11  2  9  7]
[ 5  2 10  8 12]
[12  9  8 11  5]
[ 4  7 12  5 10] to an element of Finite Field of size 13